## Variable Labeling Ratio with blunt tip.

In [1]:
from pathlib import Path
import json
import sys
sys.path.append("../../../../")

import anamic

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tifffile
from tqdm.auto import tqdm

data_dir = Path('/home/hadim/Drive/Data/Postdoc/Fake Microtubules/anamic/accuracy')
data_dir = Path('/home/hadim/local/Data/Postdoc/Fake Microtubules/anamic/accuracy')

experiment_name = 'labeling_ratio_with_blunt_tip_large'
experiment_dir = data_dir / experiment_name

In [2]:
# Get variable parameters from directory names
dirpaths = list(filter(lambda d: d.is_dir(), experiment_dir.iterdir()))
variable_parameters = [float(d.name.split('-')[1]) for d in dirpaths]
variable_parameter_name = dirpaths[0].name.split('-')[0]

# Define the worker function
def worker(image_path, variable_parameter):
    # Open the image
    image = tifffile.imread(str(image_path))

    # Load metadata
    metadata = json.load(open(image_path.with_suffix('.json')))

    # Get microtubule tip coordinates
    tip_start = np.array(metadata['tip_start'])
    tip_end = np.array(metadata['tip_end'])

    args = {}

    args['get_thick_line_args'] = {}
    args['get_thick_line_args']['length_spacing'] = 5  # pixel
    args['get_thick_line_args']['line_thickness'] = 3000 / metadata['pixel_size']  # pixel

    args['perpendicular_line_fit_args'] = {}
    args['perpendicular_line_fit_args']['length_spacing'] = 0.1  # pixel
    args['perpendicular_line_fit_args']['fit_threshold'] = 0.15
    args['perpendicular_line_fit_args']['continuous_discard'] = False

    args['offset_start'] = 4000 / metadata['pixel_size']  # pixel
    args['offset_end'] = 2000 / metadata['pixel_size']  # pixel

    args['tip_fit_args'] = {}
    args['tip_fit_args']['length_spacing'] = 0.1  # pixel
    args['tip_fit_args']['line_thickness'] = 400 / metadata["pixel_size"]  # pixel
    args['tip_fit_args']['width_spacing'] = 0.1  # pixel

    results = anamic.fitter.microtubule_tip_fitter(tip_start, tip_end, image, **args)
    line_fit_tips, x_profile, y_profile, fit_result, fit_func = results
    fit_values = fit_result.values

    # Compute x and y tip coordinates.
    mu = fit_values['mu']
    vec = line_fit_tips[1] - line_fit_tips[0]
    x, y = anamic.geometry.get_point_from_vector(vec, line_fit_tips[0], mu)

    # Save the data
    datum = {}
    datum['parameter'] = variable_parameter
    datum['image_path'] = image_path.relative_to(experiment_dir)
    datum['x'] = x * metadata['pixel_size']
    datum['y'] = y * metadata['pixel_size']
    datum['tip_start_x'] = tip_start[0] * metadata['pixel_size']
    datum['tip_start_y'] = tip_start[1] * metadata['pixel_size']
    datum['tip_end_x'] = tip_end[0] * metadata['pixel_size']
    datum['tip_end_y'] = tip_end[1] * metadata['pixel_size']
    datum['sigma'] = fit_values['sigma'] * metadata['pixel_size']
    return datum

# Generate the workers arguments
worker_args = []
for variable_parameter, dirpath in zip(variable_parameters, dirpaths):
    fnames = list(dirpath.glob('*.tif'))
    for image_path in fnames:
        worker_arg = {}
        worker_arg['image_path'] = image_path
        worker_arg['variable_parameter'] = variable_parameter
        worker_args.append(worker_arg)

In [3]:
data = anamic.utils.parallelized(worker, worker_args, args_type='dict', progress_bar=True, n_jobs=-1)
        
data = pd.DataFrame(data)
data.to_csv(experiment_dir / "analysis.csv", index=False)